In [2]:
!nvidia-smi

Mon Sep 11 16:40:07 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8               2W /  75W |      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install onnx
!pip install ultralytics==8.0.134

Paste roboflow snippet into here

In [ ]:
# !pip install roboflow
#
# from roboflow import Roboflow
# rf = Roboflow(api_key="APIKEY")
# project = rf.workspace("WORKSPACE").project("PROJECT")
# dataset = project.version(1).download("yolov8")

In [ ]:
import os

from pathlib import Path

from ultralytics import YOLO
import shutil

HOME = os.getcwd()
%cd {HOME}

challenge_dir_name = "animal-head-2310-1"
output_model_name = "animalhead2310_yolov8s.onnx"

this_dir = Path(__file__).parent
yolo_data_dir = this_dir.joinpath(challenge_dir_name)
roboflow_data_dir = this_dir.joinpath("datasets", challenge_dir_name)
yolo_inspect_path = roboflow_data_dir.joinpath("data.yaml")
yolo_best_pt_path = this_dir.joinpath("runs/detect/train/weights/best.pt")
yolo_onnx_path = this_dir.joinpath("runs/detect/train/weights/best.onnx")
model_path = this_dir.joinpath(output_model_name)

if yolo_data_dir.exists():
    shutil.move(yolo_data_dir, roboflow_data_dir)

# train
model = YOLO("yolov8s.pt")
model.train(data=str(yolo_inspect_path), epochs=300, imgsz=640)

# export to onnx
YOLO(str(yolo_best_pt_path)).export(format="onnx")

# copy to current work dir
shutil.copyfile(yolo_onnx_path, model_path)